In [16]:
import json
import pickle
import numpy as np
import random
import nltk
import tensorflow as tf

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

# Install required NLTK components
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package punkt to C:\Users\Bikram
[nltk_data]     Sarkar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Bikram
[nltk_data]     Sarkar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Bikram
[nltk_data]     Sarkar\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [17]:
# Load intents file
with open("intents.json", "r", encoding="utf-8") as f:
    intents = json.load(f)

print("Intents loaded successfully!")

Intents loaded successfully!


In [18]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        tokenized_words = nltk.word_tokenize(pattern)
        tokenized_words = [w.lower() for w in tokenized_words if w not in ignore_words]

        words.extend(tokenized_words)
        documents.append((tokenized_words, intent["tag"]))

    if intent["tag"] not in classes:
        classes.append(intent["tag"])

print(f"Documents: {len(documents)}")
print(f"Classes: {len(classes)}")
print(f"Words before cleaning: {len(words)}")

Documents: 79
Classes: 11
Words before cleaning: 266


In [19]:
# Lemmatize and sort unique words
words = sorted(list(set([lemmatizer.lemmatize(w) for w in words])))

classes = sorted(list(set(classes)))

print(f"Unique words: {len(words)}")
print(f"Unique classes: {len(classes)}")

pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

Unique words: 127
Unique classes: 11


In [20]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(w.lower()) for w in doc[0]]

    for w in words:
        bag.append(1 if w in pattern_words else 0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Shuffle + convert to arrays
random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

print("Training data created successfully!")
print("X shape:", train_x.shape)
print("Y shape:", train_y.shape)

Training data created successfully!
X shape: (79, 127)
Y shape: (79, 11)


In [21]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(len(train_x[0]),)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True, decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

model.save("chatbot_model.h5")
print("Model saved successfully as chatbot_model.h5")

Epoch 1/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1519 - loss: 2.4229  
Epoch 2/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1772 - loss: 2.3543     
Epoch 3/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2911 - loss: 2.2332 
Epoch 4/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3165 - loss: 2.1858 
Epoch 5/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2911 - loss: 2.0967 
Epoch 6/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3924 - loss: 2.0646 
Epoch 7/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4051 - loss: 1.9034 
Epoch 8/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4304 - loss: 1.8137 
Epoch 9/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4684 - loss: 1.7498 
Epoch 10/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5190 - loss: 1.5729 
Epoch 11/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6835 - loss: 1.3630 
Epoch 12/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

Model saved successfully as chatbot_model.h5


In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

model = load_model("chatbot_model.h5")
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))
intents = json.load(open("intents.json"))

def clean_up(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    return [lemmatizer.lemmatize(w.lower()) for w in sentence_words]

def bow(sentence):
    sentence_words = clean_up(sentence)
    bag = [1 if w in sentence_words else 0 for w in words]
    return np.array(bag)

def predict_class(sentence):
    bag_vec = bow(sentence)
    res = model.predict(np.array([bag_vec]))[0]
    threshold = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > threshold]
    results.sort(key=lambda x: x[1], reverse=True)

    return [{"intent": classes[i], "prob": p} for i, p in results]

def get_response(ints):
    if not ints:
        return "I didn't fully understand that. Could you rephrase?"

    tag = ints[0]["intent"]

    for intent in intents["intents"]:
        if intent["tag"] == tag:
            return random.choice(intent["responses"])

    return "Sorry, I couldn't find an answer."
    
def chatbot_response(msg):
    ints = predict_class(msg)
    return get_response(ints)

print("Chatbot engine ready!")

Chatbot engine ready!


: 